<a href="https://colab.research.google.com/github/sugampath/BERT-result/blob/master/Bertraw%20data%20accurcay%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

     |████████████████████████████████| 2.0MB 6.7MB/s 
     |████████████████████████████████| 870kB 44.7MB/s 
     |████████████████████████████████| 3.3MB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d4749e2ea31855f7c84fee01bf7ce05b704b0e82b003f2b5a15cdf7647b10ac6
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [3]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import seaborn as sns

Read the data




In [6]:
data=pd.read_csv("/content/drive/MyDrive/Thesis/dataset.csv") 

In [7]:
data.rename(columns = {"label":"category"}, inplace = True)

In [8]:
data.head(10)

,feature,category
0,सुदूरपश्चिम समाज कतारको दोस्रो साधारणसभा सम्पन...,diaspora
1,अमेरिकाबाट गायिका थापाको देशभक्ति गीत (भिडियो ...,diaspora
2,आदिवासी जनजाति मञ्च अष्ट्रेलियामा नयाँ कार्यसम...,diaspora
3,नेपाल फेस्टिवल मेलवर्नः नेपालका पर्यटनमन्त्री ...,diaspora
4,नेपाल फेष्टिबलको धन्यवाद ज्ञापन सम्पन्न\n\n१२ ...,diaspora
5,आगलागी पीडित थापालाई भेटे एनआरएन अध्यक्षले\n\n...,diaspora
6,फिजीका नेपालीलाई नेपाली भाषा पढाइने\nभीमसेन सा...,diaspora
7,अष्ट्रेलियामा नेपालीको पहिलो संस्थाले पायो नया...,diaspora
8,कराते खेलाडी ठकुरी ‘अमेरिका गट ट्यालेन्ट’मा जा...,diaspora
9,ईयूका लागि नेपाली राजदूत नियुक्त भएपछि थापाले ...,diaspora


In [9]:
data["feature"][200]

'साउदी अरबको जेद्दामा मनाइयो ल्होछार\nजेद्दा । आफ्नो मौलिक धर्म, संस्कृति र परम्परालाई जगेर्ना गर्दै निरन्तर उत्थान, सम्बर्द्धन एवं प्रबर्द्धन गर्न साउदी अरबको जेद्धा आसपासमा रहेका तामाङ श्रमिकहरूले बृहत रूपमा सोनाम ल्होछार मनाएका छन् । ज्या ल्हो अर्थात चरा बर्गको नयाँ वर्ष २८५३ को आगमनलाई हर्षोउल्लासका साथ मनाइयो ।\nनेपाल तामाङ घेदुङ प्रवास शाखा साउदी अरबका अध्यक्ष विगेन्द्रसिंह वाइवाको सभापतित्वमा सम्पन्न उक्त कार्यक्रमलाई घेदुङका महासचिव नवराज घिसिङले संचालन गरेका थिए ।\nमहावाणीज्य दूतावासका कार्यवाहक महावाणीज्यदूत रेवती रमण पौडेलको प्रमुख आतिथ्यता भएको कार्यक्रममा अन्य अतिथिहरू वाणीज्यदूत मनहरी दंगाल, श्रम सहचारी भोजराज घिमिरे, आदिबासी जनजाति महासंघ साउदी अरबका केन्द्रिय उपाध्यक्ष बालकुमार राई, आदिबासी जनजाति महासंघ क्षेत्रीय समन्वय समिति जेद्धाका अध्यक्ष ङिमा साङगे शेर्पा लगायतको उपस्थिति थियो ।\nकार्यक्रममा गैरआवासीय नेपाली संघ साउदी अरबका उपाध्यक्ष ध्रुब रिजाल, तमु धी जेद्धाका अध्यक्ष वुद्धीलाल गुरूङ, नेपाली युवा क्लब जेद्धाका अध्यक्ष कृष्ण भट्टराई, गैर आवासीय नेपाली संघ क्षेत्र

In [ ]:
#nltk.download('stopwords')

In [ ]:
stop_words = stopwords.words("nepali")
data["feature"] = data["feature"].apply(lambda x: re.sub('[।(),०-९<<?!,—,–,/,’,‘,:,\u200d,\n,\xa0]', '', x))
data["feature"] = data["feature"].apply(lambda x: " ".join([i for i in x.split()if i not in (stop_words)]))

In [ ]:
data["feature"][200]

In [ ]:
data.shape

In [11]:
data["category"].value_counts()

sports           2000
technology       2000
economy          2000
politics         2000
entertainment    2000
society          2000
diaspora         2000
health           2000
opinion          2000
international    1990
Name: category, dtype: int64

In [12]:
possible_labels = data.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'diaspora': 0,
 'economy': 1,
 'entertainment': 2,
 'health': 3,
 'international': 4,
 'opinion': 5,
 'politics': 6,
 'society': 7,
 'sports': 8,
 'technology': 9}

In [13]:
possible_labels

array(['diaspora', 'economy', 'entertainment', 'health', 'international',
       'opinion', 'politics', 'society', 'sports', 'technology'],
      dtype=object)

In [14]:
data["label"]=data.category.replace(label_dict)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

In [17]:
X_train,X_val
print(X_train.shape)
print(X_val.shape)

(13993,)
(5997,)


In [18]:
data["data_type"]=["not_set"]*data.shape[0]
data.loc[X_train,"data_type"]="train"
data.loc[X_val,"data_type"]="Val"

In [19]:
data.groupby(["category","label","data_type"]).count()

feature
category      label data_type         
diaspora      0     Val            600
                    train         1400
economy       1     Val            600
                    train         1400
entertainment 2     Val            600
                    train         1400
health        3     Val            600
                    train         1400
international 4     Val            597
                    train         1393
opinion       5     Val            600
                    train         1400
politics      6     Val            600
                    train         1400
society       7     Val            600
                    train         1400
sports        8     Val            600
                    train         1400
technology    9     Val            600
                    train         1400

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',truncation= True)

In [44]:
token_lens = []
for txt in data.feature:
  tokens = tokenizer.encode(txt, max_length=None)
  token_lens.append(len(tokens))

In [45]:
type(tokens)

list

In [46]:
print(token_lens)
print(max(token_lens))
print(min(token_lens))
print(len(token_lens))

[537, 321, 543, 492, 977, 932, 438, 715, 720, 352, 566, 1060, 521, 683, 841, 2499, 1291, 500, 1212, 1309, 615, 946, 327, 1261, 751, 441, 1610, 1680, 661, 488, 826, 632, 599, 781, 681, 535, 305, 411, 317, 2177, 1097, 362, 429, 447, 482, 769, 796, 333, 1322, 651, 395, 826, 331, 1603, 876, 1225, 146, 376, 1193, 872, 300, 315, 916, 616, 757, 676, 224, 777, 1482, 601, 611, 646, 591, 344, 623, 906, 669, 877, 2065, 648, 649, 376, 730, 712, 777, 275, 1933, 700, 1917, 825, 434, 873, 646, 783, 833, 618, 382, 409, 576, 524, 634, 925, 554, 2770, 506, 362, 319, 473, 1281, 463, 737, 571, 420, 346, 370, 660, 512, 894, 511, 352, 923, 510, 1075, 492, 727, 383, 652, 330, 507, 534, 2042, 479, 348, 1155, 451, 685, 3212, 780, 914, 378, 418, 538, 1964, 465, 360, 1067, 654, 555, 301, 350, 274, 461, 605, 383, 706, 685, 820, 164, 452, 571, 659, 4758, 769, 576, 227, 373, 357, 1561, 476, 514, 2591, 367, 571, 469, 266, 510, 1193, 339, 585, 327, 630, 194, 677, 990, 752, 318, 454, 354, 373, 433, 614, 974, 454, 556,

In [47]:
for i, token_lens in enumerate(token_lens):
  print(i, token_lens)

Streaming output truncated to the last 5000 lines.
14990 883
14991 491
14992 495
14993 287
14994 811
14995 959
14996 640
14997 430
14998 399
14999 327
15000 510
15001 2303
15002 1785
15003 528
15004 400
15005 685
15006 951
15007 646
15008 580
15009 647
15010 532
15011 148
15012 459
15013 660
15014 353
15015 629
15016 644
15017 1250
15018 481
15019 690
15020 204
15021 299
15022 391
15023 375
15024 388
15025 388
15026 1260
15027 381
15028 372
15029 486
15030 637
15031 260
15032 558
15033 546
15034 520
15035 353
15036 245
15037 906
15038 488
15039 409
15040 479
15041 582
15042 2031
15043 197
15044 554
15045 202
15046 1436
15047 450
15048 612
15049 311
15050 238
15051 1545
15052 1519
15053 501
15054 483
15055 312
15056 1301
15057 301
15058 294
15059 1433
15060 525
15061 583
15062 1118
15063 371
15064 460
15065 433
15066 520
15067 704
15068 411
15069 164
15070 512
15071 385
15072 361
15073 216
15074 409
15075 1447
15076 229
15077 409
15078 1522
15079 742
15080 754
15081 233
15082 574
15083 

In [48]:
import matplotlib.pyplot as plt

In [50]:
type(token_lens)

int

In [51]:
sns.displot(token_lens)
plt.xlim([0,2000]);
plt.xlabel('Token count');


TypeError: ignored

In [52]:
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt')


encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='Val'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt')

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='Val'].label.values)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


INPUT AND ATTENTION FORMAT


In [ ]:
b=(data["feature"][200])
print(b)
a=tokenizer.tokenize(data["feature"][200])
print(a)
print(len(a))
print(a[:512])


In [ ]:
x=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(b))
print(x)
print(len(x))

In [ ]:
encoded_data = tokenizer.encode_plus(
    text=b, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt')

input_ids = encoded_data['input_ids']
attn_mask = encoded_data['attention_mask']
print(input_ids)
print(attn_mask)


In [56]:
encoded_data_train

{'input_ids': tensor([[   101,    898,  14070,  ...,  48558,  21571,    102],
        [   101,  18263, 102116,  ...,  49362,  11208,    102],
        [   101,  29953,    886,  ...,      0,      0,      0],
        ...,
        [   101,  98440,  13718,  ...,  64566,  22599,    102],
        [   101,    884,  69897,  ...,    871,    872,    102],
        [   101,    867,  78660,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [57]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [58]:
len(dataset_train),len(dataset_val)

(13993, 5997)

In [59]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [60]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [61]:

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [62]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [63]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [65]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [67]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [68]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    FILE="model.pth"   
    torch.save(model.state_dict(),FILE)
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.6359839887076755
Validation loss: 0.3715093925442936
F1 Score (Weighted): 0.9289323445571509



In [69]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [123]:
model.load_state_dict(torch.load('/content/model.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [127]:
true_vals

array([0, 0, 0, ..., 9, 9, 9])

In [128]:
accuracy_per_class(predictions, true_vals)


Class: diaspora
Accuracy: 547/600

Class: economy
Accuracy: 560/600

Class: entertainment
Accuracy: 534/600

Class: health
Accuracy: 543/600

Class: international
Accuracy: 557/597

Class: opinion
Accuracy: 537/600

Class: politics
Accuracy: 575/600

Class: society
Accuracy: 564/600

Class: sports
Accuracy: 573/600

Class: technology
Accuracy: 581/600



In [129]:
predictions

array([[ 7.22787   , -0.7838173 , -0.17839126, ..., -0.8630043 ,
        -0.7196006 , -0.5925673 ],
       [ 7.289025  , -0.7424951 , -0.5624821 , ..., -0.74316037,
        -0.730986  , -0.6214989 ],
       [ 7.254209  , -0.66246486, -0.35974956, ..., -0.6652018 ,
        -0.8436944 , -0.5498218 ],
       ...,
       [-0.99351877, -0.560352  , -0.73075855, ..., -0.92438155,
        -0.70242715,  7.3275213 ],
       [ 6.8073387 , -0.4236522 , -0.4515408 , ..., -0.45379797,
        -1.226964  ,  0.56763417],
       [-0.94355446, -0.48126867, -0.7192316 , ..., -1.0394369 ,
        -0.6450584 ,  7.359605  ]], dtype=float32)

In [130]:
predlabel=np.argmax(predictions,axis=1)

In [131]:
predlabel

array([0, 0, 0, ..., 9, 0, 9])

In [132]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(true_vals,predlabel,digits =3))


              precision    recall  f1-score   support

           0      0.922     0.912     0.917       600
           1      0.882     0.933     0.907       600
           2      0.938     0.890     0.914       600
           3      0.927     0.905     0.916       600
           4      0.919     0.933     0.926       597
           5      0.926     0.895     0.910       600
           6      0.932     0.958     0.945       600
           7      0.934     0.940     0.937       600
           8      0.985     0.955     0.970       600
           9      0.930     0.968     0.949       600

    accuracy                          0.929      5997
   macro avg      0.929     0.929     0.929      5997
weighted avg      0.929     0.929     0.929      5997



In [134]:
print(confusion_matrix(true_vals,predlabel))

[[547  22  10   1   0   9   1   5   2   3]
 [  3 560   3   6   1   1   2  16   1   7]
 [ 12   8 534   0   6  17  14   5   1   3]
 [  1  16   3 543  12   5   5   8   1   6]
 [  1   6   1   4 557   1   4   3   1  19]
 [  7  10   8  16  10 537   9   1   0   2]
 [  1   2   7   3   2   7 575   1   2   0]
 [  8   8   0   9   2   3   4 564   1   1]
 [  9   1   2   0   8   0   3   1 573   3]
 [  4   2   1   4   8   0   0   0   0 581]]
